# High-level LSTM CNTK Example

In [1]:
import numpy as np
import os
import sys
import cntk
from cntk.layers import Embedding, LSTM, GRU, Dense, Recurrence
from cntk import sequence
from common.params_lstm import *
from common.utils import *

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.1
CNTK:  2.0
GPU:  ['Tesla M60', 'Tesla M60', 'Tesla M60', 'Tesla M60']


In [4]:
def create_symbol():
    # Weight initialiser from uniform distribution
    # Activation (unless states) is None
    with cntk.layers.default_options(init = cntk.glorot_uniform()):
        x = Embedding(EMBEDSIZE)(features) #output: list of len=BATCHSIZE of arrays with shape=(MAXLEN, EMBEDSIZE)
        x = Recurrence(GRU(NUMHIDDEN))(x) #output: list of len=BATCHSIZE of arrays with shape=(MAXLEN, NUMHIDDEN)
        x = sequence.last(x) #output: array with shape=(BATCHSIZE, NUMHIDDEN)
        x = Dense(2)(x) #output: array with shape=(BATCHSIZE, 2)
        return x

In [5]:
def init_model(m):
    # Loss (dense labels); check if support for sparse labels
    loss = cntk.cross_entropy_with_softmax(m, labels)  
    # ADAM
    #Alpha is the learning_rate
    #Beta1 is momentum parameter
    #Beta2 is variance_momentum parameter  
    learner = cntk.adam(m.parameters,
                        lr=cntk.learning_rate_schedule(LR, cntk.UnitType.minibatch) ,
                        momentum=cntk.momentum_schedule(BETA_1), 
                        variance_momentum=cntk.momentum_schedule(BETA_2),
                        epsilon=EPS,
                        use_mean_gradient=False, #if True, error: useMeanGradient should not be used with per-minibatch learning rate setting
                        unit_gain=True)
    trainer = cntk.Trainer(m, (loss, cntk.classification_error(m, labels)), [learner])
    return trainer

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES, one_hot=True)# CNTK format
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Trimming to 150000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000, 2) (25000, 2)
int32 int32 float32 float32
CPU times: user 6.07 s, sys: 360 ms, total: 6.43 s
Wall time: 7.47 s


In [7]:
%%time
# Placeholders
features = sequence.input_variable(shape=MAXFEATURES, is_sparse=True)
labels = cntk.input_variable(2)

# Load symbol
sym = create_symbol()

print(features)
print(labels)

Input('Input3', [#, *], [150000])
Input('Input4', [#], [2])
CPU times: user 40 ms, sys: 608 ms, total: 648 ms
Wall time: 969 ms


In [8]:
%%time
trainer = init_model(sym)

CPU times: user 284 ms, sys: 16 ms, total: 300 ms
Wall time: 301 ms


In [9]:
%%time
# Train model
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        data_1hot = cntk.Value.one_hot(data, MAXFEATURES)#TODO: do this externally and generate batches of 1hot
        #print(data.shape) # (BATCHSIZE, MAXLEN)
        #print(data_1hot.shape) (BATCHSIZE, MAXLEN, MAXFEATURES)
        trainer.train_minibatch({features: data_1hot, labels: label})

    # Log (this is just last batch in epoch, not average of batches)
    eval_error = trainer.previous_minibatch_evaluation_average
    print("Epoch %d  |  Accuracy: %.6f" % (j+1, (1-eval_error)))

Epoch 1  |  Accuracy: 0.437500
Epoch 2  |  Accuracy: 0.765625
Epoch 3  |  Accuracy: 0.843750
CPU times: user 57.6 s, sys: 2.12 s, total: 59.8 s
Wall time: 59.7 s


In [10]:
%%time
# Predict and then score accuracy
# Apply softmax since that is only applied at training
# with cross-entropy loss
z = cntk.softmax(sym)
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = np.argmax(y_test[:n_samples], axis=-1)
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    data = cntk.Value.one_hot(data, MAXFEATURES)
    predicted_label_probs = z.eval({features : data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = np.argmax(predicted_label_probs, axis=-1)
    c += 1

CPU times: user 7.83 s, sys: 164 ms, total: 8 s
Wall time: 8.07 s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.822636217949
